In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.01
num_epoch = 10

In [2]:
mnist_train = dset.MNIST('./', train= True, transform = transforms.ToTensor(), target_transform=None, download= True)
mnist_test = dset.MNIST('./', train= False, transform = transforms.ToTensor(), target_transform=None, download= True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size= batch_size, shuffle=True, num_workers= 2, drop_last = True)

test_loader = torch.utils.data.DataLoader(mnist_test, batch_size= batch_size, shuffle=False, num_workers= 2, drop_last = True)

0it [00:00, ?it/s]

9920512it [00:01, 8124923.61it/s]                            


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 122858.92it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


1654784it [00:00, 2007371.94it/s]                            
0it [00:00, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


8192it [00:00, 46442.94it/s]            


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1,16,5),
        nn.ReLU(),
        nn.Conv2d(16,32,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLU(),
        nn.Linear(100, 10)

    )
  def forward(self,x):
    out = self.layer(x)
    out = out.view(batch_size, -1)
    out = self.fc_layer(out)
    return out

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [5]:
device

device(type='cuda', index=0)

In [6]:
loss_arr = []
for i in range(num_epoch):
  for j, [image,label] in enumerate(train_loader):
    x = image.to(device)
    y = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y)
    loss.backward()
    optimizer.step()

    if j % 1000 == 0:
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3023, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0962, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0106, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0055, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0142, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0077, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0453, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0280, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0091, device='cuda:0', grad_fn=<NllLossBackward>)


In [7]:
correct = 0
total = 0
with torch.no_grad():
  for image, label in test_loader:
    x = image.to(device)
    y = label.to(device)

    output = model.forward(x)
    _, output_index = torch.max(output,1)

    total += label.size(0)
    correct += (output_index == y).sum().float()
  print("accuracy : {}".format(100*correct/total))

accuracy : 98.96835327148438
